# Prepare tutorial dataset from ATLAS Open Data

In [1]:

#!pip install uproot
import uproot
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # to see all columns of df.head()
import time


In [2]:
#load signal, background data
# ATLAS 13 TeV simulation
#More info on dataset  : http://opendata.atlas.cern/release/2020/documentation/datasets/mc.html

nevents=200000  # limit number of events to treat
#nevents=1E10 

inputdatapath="/Users/rousseau/DocumentsLourds/OpenData/CERNOpenData/dataWW_ATLAS_openData13TeV/"
outputdatapath="/Users/rousseau/Downloads/"

#detaillevel="d0"
detaillevel="d1"


# chose one of the file to be filtered
outkey="qq"
outkey="ggH"
outkey="VBFH"

# http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_363492.llvv.2lep.root 
if outkey=="ggH":
    inputfilename=inputdatapath+"mc_345324.ggH125_WW2lep.2lep.root"
    outputfilename=outputdatapath+"dataWW_ggH"+"_"+detaillevel+".csv.gz" 
elif outkey=="VBFH":
    inputfilename=inputdatapath+"mc_345323.VBFH125_WW2lep.2lep.root"
    outputfilename=outputdatapath+"dataWW_VBFH"+"_"+detaillevel+".csv.gz" 

elif outkey=="qq":
    inputfilename=inputdatapath+"mc_363492.llvv.2lep.root"
    outputfilename=outputdatapath+"dataWW_qq"+"_"+detaillevel+".csv.gz" 


print ("Will read file:",inputfilename)
print ("Will write file",outputfilename)
tree = uproot.open(inputfilename)  


    
#http://opendata.cern.ch/eos/opendata/atlas/OutreachDatasets/2020-01-22/2lep/MC/mc_345323.VBFH125_WW2lep.2lep.root
#tree = uproot.open("dataWW/mc_161005.ggH125_WW2lep.root") ; outhdf="dataWW/dataWW_ggH.hdf"; outkey="ggH" # http://opendata.cern.ch/record/3825

#tree = uproot.open("dataWW/mc_161055.VBFH125_WW2lep.root"); outhdf="dataWW/dataWW_VBFH.hdf"; outkey="VBFH" # http://opendata.cern.ch/record/3826






Will read file: /Users/rousseau/DocumentsLourds/OpenData/CERNOpenData/dataWW_ATLAS_openData13TeV/mc_345323.VBFH125_WW2lep.2lep.root
Will write file /Users/rousseau/Downloads/dataWW_VBFH_d1.csv.gz


In [3]:
roottree=tree['mini']
roottree.show()
roottree.keys()
print ("Number of entries in the tree:",roottree.numentries)

runNumber                  (no streamer)              asdtype('>i4')
eventNumber                (no streamer)              asdtype('>i4')
channelNumber              (no streamer)              asdtype('>i4')
mcWeight                   (no streamer)              asdtype('>f4')
scaleFactor_PILEUP         (no streamer)              asdtype('>f4')
scaleFactor_ELE            (no streamer)              asdtype('>f4')
scaleFactor_MUON           (no streamer)              asdtype('>f4')
scaleFactor_PHOTON         (no streamer)              asdtype('>f4')
scaleFactor_TAU            (no streamer)              asdtype('>f4')
scaleFactor_BTAG           (no streamer)              asdtype('>f4')
scaleFactor_LepTRIGGER     (no streamer)              asdtype('>f4')
scaleFactor_PhotonTRIGGER  (no streamer)              asdtype('>f4')
trigE                      (no streamer)              asdtype('bool')
trigM                      (no streamer)              asdtype('bool')
trigP                      (no s

In [4]:
roottree.values

<bound method TTreeMethods.values of <TTree b'mini' at 0x0001164d50d0>>

In [5]:
#full list of variables http://opendata.atlas.cern/release/2020/documentation/datasets/dataset13.html

extract_vars_scalar=['runNumber', 'eventNumber', 'channelNumber', 'mcWeight',
       'scaleFactor_PILEUP', 'scaleFactor_ELE', 'scaleFactor_MUON',
       'scaleFactor_PHOTON', 'scaleFactor_TAU', 'scaleFactor_BTAG',
       'scaleFactor_LepTRIGGER', 'scaleFactor_PhotonTRIGGER', 'trigE', 'trigM',
       'trigP', 'lep_n', 'met_et',
       'met_phi', 'jet_n', 'photon_n',
       'tau_n', "ditau_m",  
       'met_et_syst',
       'XSection', 'SumWeights', 'largeRjet_n']

extract_vars_array=['lep_truthMatched', 'lep_trigMatched', 'lep_pt',
       'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_charge', 'lep_type',
       'lep_isTightID', 'lep_ptcone30', 'lep_etcone20',
       'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 
        'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_jvt',
       'jet_trueflav', 'jet_truthMatched', 'jet_MV2c10', 
       'photon_truthMatched', 'photon_trigMatched', 'photon_pt', 'photon_eta',
       'photon_phi', 'photon_E', 'photon_isTightID', 'photon_ptcone30',
       'photon_etcone20', 'photon_convType', 'tau_pt', 'tau_eta',
       'tau_phi', 'tau_E', 'tau_isTightID', 'tau_truthMatched',
       'tau_trigMatched', 'tau_nTracks', 'tau_BDTid',  'lep_pt_syst',
        'jet_pt_syst', 'photon_pt_syst', 'tau_pt_syst',
       'largeRjet_pt',
       'largeRjet_eta', 'largeRjet_phi', 'largeRjet_E', 'largeRjet_m',
       'largeRjet_truthMatched', 'largeRjet_D2', 'largeRjet_tau32',
       'largeRjet_pt_syst', 'tau_charge']

# strict minimum just MET and the lepton 4 mom
if detaillevel=="d0":
    extract_vars_scalar=['runNumber', 'eventNumber', 'channelNumber', 'mcWeight',
        'lep_n', 'met_et',
       'met_phi',
       'XSection', 'SumWeights']
    extract_vars_array=['lep_pt',
       'lep_eta', 'lep_phi','lep_charge', 'lep_type']



if detaillevel=="d1":
    extract_vars_scalar=['runNumber', 'eventNumber', 'channelNumber', 'mcWeight',
        'lep_n', 'met_et',
       'met_phi','jet_n',
       'XSection', 'SumWeights']

    extract_vars_array=['lep_pt',
       'lep_eta', 'lep_phi','lep_E','lep_charge', 'lep_type',
       'jet_pt', 'jet_eta', 'jet_phi', 'jet_E']






extract_vars=extract_vars_scalar+extract_vars_array

In [6]:
startcpu = time.process_time()
startwallclock = time.time()


#few var to debug
#extract_vars=[  'runNumber',  'eventNumber',  'channelNumber', 'mcWeight','lep_pt','jet_pt']
#flatten=False mandatory if some variable size array
# FIXME uproot pandas.df is real slow. Should move to numpy reading.
df=roottree.pandas.df(extract_vars,flatten=False,entrystop=nevents) 
#df=roottree.pandas.df(extract_vars,flatten=False,entrystart=35000,entrystop=65000) 

#df.keys() # to have the list of variable
display(df.head())
display(df.describe)

print("CPU time : ", time.process_time() - startcpu)
print("Wall clock time ",time.time() - startwallclock)



,runNumber,eventNumber,channelNumber,mcWeight,lep_n,met_et,met_phi,jet_n,XSection,SumWeights,lep_pt,lep_eta,lep_phi,lep_E,lep_charge,lep_type,jet_pt,jet_eta,jet_phi,jet_E
entry,,,,,,,,,,,,,,,,,,,,
0,284500,384,345323,3.749917,2,116861.976562,-1.265381,2,1.905877,4389990.0,"[57986.78, 33692.395]","[-0.17389055, -0.87548834]","[-1.4897541, -1.6281896]","[58865.79, 47450.777]","[-1, 1]","[13, 11]","[29894.691, 29083.451]","[-2.4546714, 1.2145077]","[-2.4570606, 0.5750098]","[175423.42, 53506.61]"
1,284500,841,345323,3.749917,2,112019.656250,-0.868992,3,1.905877,4389990.0,"[29739.217, 12656.194]","[-1.0223949, 0.05334374]","[0.9359129, 0.62833863]","[46684.273, 12674.6455]","[1, -1]","[11, 13]","[99282.52, 32057.973, 21531.564]","[-2.3501935, 0.7605696, -1.1584153]","[3.121195, -1.5141237, 1.8501498]","[525478.25, 42347.35, 37972.08]"
2,284500,184,345323,3.749917,2,314240.406250,-0.370740,3,1.905877,4389990.0,"[42921.27, 18685.36]","[-0.8960725, -1.1407573]","[-0.552366, -1.6429868]","[61337.41, 32220.232]","[1, -1]","[13, 11]","[243037.17, 73487.56, 27080.62]","[1.2937653, -2.0674562, 2.209404]","[2.7463968, -2.9270153, 1.278851]","[476635.8, 295285.38, 124964.766]"
3,284500,817,345323,3.749917,2,72023.109375,-0.991769,2,1.905877,4389990.0,"[67622.86, 65901.164]","[0.9023578, 0.62570417]","[-0.0453637, -0.4605257]","[97073.4, 79227.984]","[-1, 1]","[13, 13]","[213954.38, 23830.812]","[2.499094, 2.4734304]","[2.9268243, 0.578938]","[1311011.4, 142503.3]"
4,284500,1588,345323,3.749917,2,4425.194824,-2.432950,0,1.905877,4389990.0,"[40542.984, 21005.13]","[0.4464862, 0.29136705]","[1.6497122, -0.47812402]","[44651.688, 21903.324]","[1, -1]","[11, 13]",[],[],[],[]


<bound method NDFrame.describe of         runNumber  eventNumber  channelNumber  mcWeight  lep_n         met_et  \
entry                                                                           
0          284500          384         345323  3.749917      2  116861.976562   
1          284500          841         345323  3.749917      2  112019.656250   
2          284500          184         345323  3.749917      2  314240.406250   
3          284500          817         345323  3.749917      2   72023.109375   
4          284500         1588         345323  3.749917      2    4425.194824   
...           ...          ...            ...       ...    ...            ...   
199995     284500       197164         345323  3.749917      2   27219.941406   
199996     284500       197053         345323  3.749917      2   34253.167969   
199997     284500       198078         345323  3.749917      2   77188.273438   
199998     284500       198428         345323  3.749917      2   79800.8750

CPU time :  3455.015268
Wall clock time  1760.8257808685303


In [7]:
#display(df.head(100))
allvars=df.columns.tolist() #output columns into list
#check we have correctly listed the scalar and vectors
if (sorted(extract_vars)!=sorted(allvars)):
    print ("Explicit list :", sorted(extract_vars))
    print ("Complete list :", sorted(allvars))

    
    raise Exception("some missing variables! Please fix extract_vars_scalar or extract_vars_array.")

In [8]:
# now keep only the first two elements of each array, with variable name like lep_pt_0 lpt_pt_1
dfflat=df.drop(columns=extract_vars_array)
display(dfflat.head())
#extract_vars_array_debug=["jet_pt","photon_pt"] #truthMatched"] # for debugging

for var in extract_vars_array:
    # print ("treating var=",var)
    dff=pd.DataFrame( df[var].values.tolist())
    #display(dff.head())


    for i in range (2,len(dff.columns)):
        dff=dff.drop(columns=[i]) # only keep first two columns
        
    # if necessary complete up to two columns
    if len(dff.columns)<2:
        dff["add0"]=np.nan
    
    if len(dff.columns)<2:
        dff["add1"]=np.nan

        
    colunames=[]
    for i in range(2):
        colunames+=[var+"_"+str(i)]
    dff.columns=colunames #give names to the two elements

    dfflat=pd.concat([dfflat,dff],axis=1) # merge in


dfflat.head()


,runNumber,eventNumber,channelNumber,mcWeight,lep_n,met_et,met_phi,jet_n,XSection,SumWeights
entry,,,,,,,,,,
0,284500,384,345323,3.749917,2,116861.976562,-1.265381,2,1.905877,4389990.0
1,284500,841,345323,3.749917,2,112019.656250,-0.868992,3,1.905877,4389990.0
2,284500,184,345323,3.749917,2,314240.406250,-0.370740,3,1.905877,4389990.0
3,284500,817,345323,3.749917,2,72023.109375,-0.991769,2,1.905877,4389990.0
4,284500,1588,345323,3.749917,2,4425.194824,-2.432950,0,1.905877,4389990.0


,runNumber,eventNumber,channelNumber,mcWeight,lep_n,met_et,met_phi,jet_n,XSection,SumWeights,lep_pt_0,lep_pt_1,lep_eta_0,lep_eta_1,lep_phi_0,lep_phi_1,lep_E_0,lep_E_1,lep_charge_0,lep_charge_1,lep_type_0,lep_type_1,jet_pt_0,jet_pt_1,jet_eta_0,jet_eta_1,jet_phi_0,jet_phi_1,jet_E_0,jet_E_1
0,284500,384,345323,3.749917,2,116861.976562,-1.265381,2,1.905877,4389990.0,57986.781250,33692.394531,-0.173891,-0.875488,-1.489754,-1.628190,58865.789062,47450.777344,-1,1,13,11,29894.691406,29083.451172,-2.454671,1.214508,-2.457061,0.575010,1.754234e+05,53506.609375
1,284500,841,345323,3.749917,2,112019.656250,-0.868992,3,1.905877,4389990.0,29739.216797,12656.194336,-1.022395,0.053344,0.935913,0.628339,46684.273438,12674.645508,1,-1,11,13,99282.523438,32057.972656,-2.350194,0.760570,3.121195,-1.514124,5.254782e+05,42347.351562
2,284500,184,345323,3.749917,2,314240.406250,-0.370740,3,1.905877,4389990.0,42921.269531,18685.359375,-0.896073,-1.140757,-0.552366,-1.642987,61337.410156,32220.232422,1,-1,13,11,243037.171875,73487.562500,1.293765,-2.067456,2.746397,-2.927015,4.766358e+05,295285.375000
3,284500,817,345323,3.749917,2,72023.109375,-0.991769,2,1.905877,4389990.0,67622.859375,65901.164062,0.902358,0.625704,-0.045364,-0.460526,97073.398438,79227.984375,-1,1,13,13,213954.375000,23830.812500,2.499094,2.473430,2.926824,0.578938,1.311011e+06,142503.296875
4,284500,1588,345323,3.749917,2,4425.194824,-2.432950,0,1.905877,4389990.0,40542.984375,21005.130859,0.446486,0.291367,1.649712,-0.478124,44651.687500,21903.324219,1,-1,11,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# now put some columns at the end to improve visibility

varlist=dfflat.columns.tolist()
for aname in ["mcWeight","runNumber","XSection","SumWeights"]:
    if not aname in varlist:
        continue
    varlist.remove(aname)
    varlist.append(aname)
    
    #varlist.insert(varlist.index("channelNumber")+1,aname)

vnames=[]
for s in varlist: # find all the array number e.g. lep_n
    i=s.find("_n")
    if i>=0 and i==len(s)-2:
        vnames+=[s]

#print(vnames)

for vname in vnames:
    varlist.remove(vname) # remove e.g. lep_n
    aname=vname[:-2]
    found=False
    for i,s in enumerate(varlist): # finds the first e.g. lep_blah
        if s.startswith(aname) :
            found=True
            break
    if found:
        varlist.insert(i,vname) # insert e.g. lep_n before lep_blah


# check we have only reordered
if sorted(dfflat.columns.tolist())!=sorted(varlist):
    print ("before:",sorted(dfflat.columns.tolist()) )
    print ("after :",sorted(varlist) )
    raise Exception("something wrong when reordering the columns!")

# do the reordering
dfflat = dfflat.reindex(columns=varlist)
display(dfflat.head())
display(dfflat.describe())



,eventNumber,channelNumber,met_et,met_phi,lep_n,lep_pt_0,lep_pt_1,lep_eta_0,lep_eta_1,lep_phi_0,lep_phi_1,lep_E_0,lep_E_1,lep_charge_0,lep_charge_1,lep_type_0,lep_type_1,jet_n,jet_pt_0,jet_pt_1,jet_eta_0,jet_eta_1,jet_phi_0,jet_phi_1,jet_E_0,jet_E_1,mcWeight,runNumber,XSection,SumWeights
0,384,345323,116861.976562,-1.265381,2,57986.781250,33692.394531,-0.173891,-0.875488,-1.489754,-1.628190,58865.789062,47450.777344,-1,1,13,11,2,29894.691406,29083.451172,-2.454671,1.214508,-2.457061,0.575010,1.754234e+05,53506.609375,3.749917,284500,1.905877,4389990.0
1,841,345323,112019.656250,-0.868992,2,29739.216797,12656.194336,-1.022395,0.053344,0.935913,0.628339,46684.273438,12674.645508,1,-1,11,13,3,99282.523438,32057.972656,-2.350194,0.760570,3.121195,-1.514124,5.254782e+05,42347.351562,3.749917,284500,1.905877,4389990.0
2,184,345323,314240.406250,-0.370740,2,42921.269531,18685.359375,-0.896073,-1.140757,-0.552366,-1.642987,61337.410156,32220.232422,1,-1,13,11,3,243037.171875,73487.562500,1.293765,-2.067456,2.746397,-2.927015,4.766358e+05,295285.375000,3.749917,284500,1.905877,4389990.0
3,817,345323,72023.109375,-0.991769,2,67622.859375,65901.164062,0.902358,0.625704,-0.045364,-0.460526,97073.398438,79227.984375,-1,1,13,13,2,213954.375000,23830.812500,2.499094,2.473430,2.926824,0.578938,1.311011e+06,142503.296875,3.749917,284500,1.905877,4389990.0
4,1588,345323,4425.194824,-2.432950,2,40542.984375,21005.130859,0.446486,0.291367,1.649712,-0.478124,44651.687500,21903.324219,1,-1,11,13,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.749917,284500,1.905877,4389990.0


,eventNumber,channelNumber,met_et,met_phi,lep_n,lep_pt_0,lep_pt_1,lep_eta_0,lep_eta_1,lep_phi_0,lep_phi_1,lep_E_0,lep_E_1,lep_charge_0,lep_charge_1,lep_type_0,lep_type_1,jet_n,jet_pt_0,jet_pt_1,jet_eta_0,jet_eta_1,jet_phi_0,jet_phi_1,jet_E_0,jet_E_1,mcWeight,runNumber,XSection,SumWeights
count,2.000000e+05,200000.0,2.000000e+05,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,1.588580e+05,71974.000000,158858.000000,71974.000000,158858.000000,71974.000000,1.588580e+05,7.197400e+04,200000.000000,200000.0,200000.000000,2.000000e+05
mean,5.618368e+05,345323.0,6.125444e+04,-0.006217,2.006050,5.774107e+04,25521.282278,0.002152,0.002656,0.002530,0.009040,1.056596e+05,48120.910743,-0.002700,0.003400,11.908710,11.961920,1.326380,9.105082e+04,45860.138548,0.006987,0.003688,-0.006843,0.005130,2.361052e+05,1.262624e+05,3.737287,284500.0,1.906240,4.390892e+06
std,3.208474e+05,0.0,4.510286e+04,1.812713,0.077996,3.359358e+04,15738.557564,1.158605,1.187890,1.810775,1.814271,8.409279e+04,42408.648203,0.999999,0.999997,0.995827,0.999277,1.058495,6.827779e+04,31418.373602,1.533908,1.572302,1.814658,1.812236,2.320534e+05,1.199018e+05,0.308322,0.0,0.000364,9.020276e+02
min,2.000000e+00,345323.0,9.562187e+01,-3.141569,2.000000,2.500052e+04,7000.291992,-2.696373,-2.699926,-3.141549,-3.141536,2.502267e+04,7003.763184,-1.000000,-1.000000,11.000000,11.000000,0.000000,2.000121e+04,20000.445312,-2.499998,-2.499821,-3.141566,-3.141570,2.025917e+04,2.020270e+04,-3.749917,284500.0,1.905877,4.389990e+06
25%,3.277008e+05,345323.0,3.325170e+04,-1.576492,2.000000,3.705454e+04,14427.725586,-0.848496,-0.892411,-1.567547,-1.563081,5.235055e+04,21771.405762,-1.000000,-1.000000,11.000000,11.000000,1.000000,4.418742e+04,25255.976562,-1.396042,-1.437151,-1.581503,-1.568343,8.780352e+04,4.879466e+04,3.749917,284500.0,1.905877,4.389990e+06
50%,5.014455e+05,345323.0,5.191154e+04,-0.018328,2.000000,4.925617e+04,22259.882812,0.004078,0.006082,0.014238,0.017808,7.927565e+04,35129.060547,-1.000000,1.000000,11.000000,11.000000,1.000000,7.305882e+04,34536.884766,0.013694,0.004739,-0.009425,0.018331,1.607762e+05,8.963575e+04,3.749917,284500.0,1.905877,4.389990e+06
75%,7.697402e+05,345323.0,7.690627e+04,1.560394,2.000000,6.838680e+04,31861.653809,0.855856,0.897754,1.570543,1.577133,1.295045e+05,58666.938477,1.000000,1.000000,13.000000,13.000000,2.000000,1.159637e+05,55887.931641,1.413900,1.448666,1.565018,1.568228,3.002983e+05,1.564262e+05,3.749917,284500.0,1.905877,4.389990e+06
max,1.199997e+06,345323.0,3.634596e+06,3.141551,4.000000,3.624608e+06,382168.281250,2.699811,2.699214,3.141534,3.141544,3.624717e+06,870747.625000,1.000000,1.000000,13.000000,13.000000,10.000000,1.658162e+06,586593.187500,2.499952,2.499937,3.141559,3.141583,4.006936e+06,2.388901e+06,3.749917,284500.0,1.905877,4.389990e+06


In [10]:
# convert momentum and energy to GeV
for togev in ["met_et","lep_pt_0","lep_pt_1","jet_pt_0","jet_pt_1"]:
    if togev in dfflat.keys():
        dfflat[togev]/=1000.


In [11]:
dfflat.fillna(-7,inplace=True) # replace NaN by unphysical real value
dfflat.head()
print ("number of events",len(dfflat.index))
print ("total weight of these events",dfflat.mcWeight.sum())
print ("channelNumber",dfflat["channelNumber"][0])
print ("SumWeights",dfflat["SumWeights"][0])
print ("XSection",dfflat["XSection"][0])

number of events 200000
total weight of these events 747440.75
channelNumber 345323
SumWeights 4389990.0
XSection 1.9058765


In [12]:
dfflat.describe()

,eventNumber,channelNumber,met_et,met_phi,lep_n,lep_pt_0,lep_pt_1,lep_eta_0,lep_eta_1,lep_phi_0,lep_phi_1,lep_E_0,lep_E_1,lep_charge_0,lep_charge_1,lep_type_0,lep_type_1,jet_n,jet_pt_0,jet_pt_1,jet_eta_0,jet_eta_1,jet_phi_0,jet_phi_1,jet_E_0,jet_E_1,mcWeight,runNumber,XSection,SumWeights
count,2.000000e+05,200000.0,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,2.000000e+05,200000.000000,200000.0,200000.000000,2.000000e+05
mean,5.618368e+05,345323.0,61.254960,-0.006217,2.006050,57.741066,25.521282,0.002152,0.002656,0.002530,0.009040,1.056596e+05,48120.910743,-0.002700,0.003400,11.908710,11.961920,1.326380,70.880787,12.022778,-1.434421,-4.479583,-1.445405,-4.479064,1.875346e+05,4.543356e+04,3.737287,284500.0,1.906240,4.390892e+06
std,3.208474e+05,0.0,45.104282,1.812713,0.077996,33.593580,15.738558,1.158605,1.187890,1.810775,1.814271,8.409279e+04,42408.648203,0.999999,0.999997,0.995827,0.999277,1.058495,72.620436,31.605592,3.145023,3.491334,3.256724,3.533596,2.277730e+05,9.405599e+04,0.308322,0.0,0.000364,9.020276e+02
min,2.000000e+00,345323.0,0.095622,-3.141569,2.000000,25.000521,7.000292,-2.696373,-2.699926,-3.141549,-3.141536,2.502267e+04,7003.763184,-1.000000,-1.000000,11.000000,11.000000,0.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000e+00,-7.000000e+00,-3.749917,284500.0,1.905877,4.389990e+06
25%,3.277008e+05,345323.0,33.251699,-1.576492,2.000000,37.054540,14.427726,-0.848496,-0.892411,-1.567547,-1.563081,5.235055e+04,21771.405762,-1.000000,-1.000000,11.000000,11.000000,1.000000,24.793918,-7.000000,-2.275491,-7.000000,-2.790456,-7.000000,4.011967e+04,-7.000000e+00,3.749917,284500.0,1.905877,4.389990e+06
50%,5.014455e+05,345323.0,51.911537,-0.018328,2.000000,49.256170,22.259883,0.004078,0.006082,0.014238,0.017808,7.927565e+04,35129.060547,-1.000000,1.000000,11.000000,11.000000,1.000000,57.474021,-7.000000,-0.771714,-7.000000,-0.828875,-7.000000,1.193684e+05,-7.000000e+00,3.749917,284500.0,1.905877,4.389990e+06
75%,7.697402e+05,345323.0,76.906273,1.560394,2.000000,68.386799,31.861654,0.855856,0.897754,1.570543,1.577133,1.295045e+05,58666.938477,1.000000,1.000000,13.000000,13.000000,2.000000,101.699098,26.742459,1.089902,-1.168697,1.157947,-1.226929,2.514305e+05,5.723328e+04,3.749917,284500.0,1.905877,4.389990e+06
max,1.199997e+06,345323.0,3634.595750,3.141551,4.000000,3624.607500,382.168281,2.699811,2.699214,3.141534,3.141544,3.624717e+06,870747.625000,1.000000,1.000000,13.000000,13.000000,10.000000,1658.162000,586.593187,2.499952,2.499937,3.141559,3.141583,4.006936e+06,2.388901e+06,3.749917,284500.0,1.905877,4.389990e+06


In [13]:
dfflat.to_csv(outputfilename,float_format='%.5g',index = False)# do csv file